In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from Utils.Solver import *
from Utils.Signals import *

In [9]:
# Let's assume we are interested in the following stocks: AAPL, MSFT, TSLA, AMZN, GOOG
tickers = ['AAPL', 'MSFT', 'TSLA', 'AMZN', 'GOOG', 'IBM', 'NFLX', 'NVDA', 'AMD', 'INTC', 'TXN', 'QCOM', 'MU', 'ADI', 'HPQ', 'GLW', 'AVGO', 'ADP', 'INTU', 'PFE'] 
tickers = ['AAPL', 'MSFT', 'TSLA', 'AMZN', 'GOOG']
aaa_game_company_tickers = [
    "MSFT",       # Activision Blizzard (via Microsoft)
    "7832.T",     # Bandai Namco Holdings
    "9697.T",     # Capcom
    "CDR.WA",     # CD Projekt
    "EA",         # Electronic Arts
    "EMBRAC-B.ST",   # Embracer Group
    "7974.T",     # Nintendo
    "9684.T",     # Square Enix
    "TTWO",       # Take-Two Interactive
    "UBI.PA",     # Ubisoft Entertainment
    "SONY",       # Sony Group Corporation
    "6460.T",     # Sega Sammy Holdings
    "9766.T",     # Konami Holdings
]

aa_game_company_tickers = [
    "3659.T",     # Nexon
    "PDX.ST",     # Paradox Interactive
    "259960.KQ",  # KRAFTON
    "036570.KS",  # NCSoft
    "FDEV.L",     # Frontier Developments
    "REMEDY.HE",  # Remedy Entertainment
    "NACON.PA",   # Nacon
    "SF.ST",      # Stillfront Group
]

game_company_tickers = aaa_game_company_tickers + aa_game_company_tickers

# Download historical stock data
data = yf.download(game_company_tickers, start='2010-01-01', end='2025-01-01')
# Initialize the portfolio solver with appropriate penalty and max weight threshold
portfolio_solver = Portfolio_Solver(0.8, max_weight_threshold=0.2 )

[*********************100%***********************]  21 of 21 completed


In [9]:
start_date_signal = '2000-01-01'
end_date_signal = '2021-01-01'
date_range_signal = pd.date_range(start=start_date_signal, end=end_date_signal)

start_date_eval = '2019-01-01'
end_date_eval = '2020-01-01'
date_range_eval = pd.date_range(start=start_date_eval, end=end_date_eval)

display(date_range_eval)

DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
               '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08',
               '2019-01-09', '2019-01-10',
               ...
               '2019-12-23', '2019-12-24', '2019-12-25', '2019-12-26',
               '2019-12-27', '2019-12-28', '2019-12-29', '2019-12-30',
               '2019-12-31', '2020-01-01'],
              dtype='datetime64[ns]', length=366, freq='D')

In [ ]:
# Initialize an empty list to store the rows for the first DataFrame
dataset_scores = []

# Step 1: Create the DataFrame with rsi_scores, macd_scores, and sma_scores
for date in date_range_signal:

    # Step 1a: Calculate the RSI signal scores
    rsi_signal_scores = calculate_rsi_signal(data, tickers, date=date, period=14)
    rsi_scores = np.array([score[1] for score in rsi_signal_scores])

    # Step 1b: Calculate the MACD signal scores
    macd_signal_scores = calculate_macd_signal(data, tickers, date=date)
    macd_scores = np.array([score[1] for score in macd_signal_scores])

    # Step 1c: Calculate the SMA signal scores
    sma_signal_scores = calculate_sma_signal(data, tickers, date=date)
    sma_scores = np.array([score[1] for score in sma_signal_scores])

    print(f"processing {date} : {rsi_scores}, {macd_scores}, {sma_scores} ")

    if np.any(np.isnan(rsi_scores)) or np.any(np.isnan(macd_scores)) or np.any(np.isnan(sma_scores)):
        #print(f"Skipping {date} due to NaN values in the signals.")
        continue  # Skip this date and move to the next one


    # Step 1d: Add the scores to the first dataset
    dataset_scores.append({
        'date': date,
        'rsi_scores': rsi_scores,
        'macd_scores': macd_scores,
        'sma_scores': sma_scores
    })


# Convert the first dataset into a DataFrame
df_scores = pd.DataFrame(dataset_scores)
display(df_scores.head)

processing 2000-01-01 00:00:00 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan] 
processing 2000-01-02 00:00:00 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan] 
processing 2000-01-03 00:00:00 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan] 
processing 2000-01-04 00:00:00 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan

KeyboardInterrupt: 

In [5]:
start_date_train = '2018-01-01'
end_date_train = '2019-01-01'
date_range_train = pd.date_range(start=start_date_train, end=end_date_train)

start_date_eval = '2020-01-01'
end_date_eval = '2021-01-01'
date_range_eval = pd.date_range(start=start_date_eval, end=end_date_eval)

# Filter the dataframe within the date range
df_train = df_scores[(df_scores['date'] >= start_date_train) & (df_scores['date'] <= end_date_train)]
display(df_train)
df_eval = df_scores[(df_scores['date'] >= start_date_eval) & (df_scores['date'] <= end_date_eval)]
display(df_eval)

,date,rsi_scores,macd_scores,sma_scores
1692,2018-01-02,"[52.03588522479302, 59.83736445276174, 48.9947...","[-0.11895414938635707, -0.08168810864991893, -...","[3.485084838867188, 9.17165859222412, -0.80163..."
1693,2018-01-03,"[51.95032550268461, 62.078039340088495, 46.764...","[-0.0978985204938759, -0.05676588897711521, -0...","[3.524692077636722, 9.214494056701653, -0.8572..."
1694,2018-01-04,"[54.11756783981267, 65.96336836819373, 44.9957...","[-0.0693594057895649, -0.0006584131729289178, ...","[3.5656105232238744, 9.264214000701898, -0.908..."
1695,2018-01-05,"[59.01943586781675, 70.57682208048138, 46.6161...","[-0.019944594622849976, 0.09090460888646656, -...","[3.6109470367431697, 9.332059116363524, -0.956..."
1696,2018-01-08,"[56.86107649172263, 70.93037356977361, 59.5878...","[0.0007710309159785766, 0.14137003470394083, 0...","[3.656642608642578, 9.404647769927976, -0.9697..."
...,...,...,...,...
1938,2018-12-24,"[22.98356456001271, 29.476189774864377, 31.334...","[-0.2733462909610922, -1.256426979148671, -0.5...","[-0.3894151306152338, 3.6528398132324185, 1.21..."
1939,2018-12-26,"[36.49253698394841, 43.11257702824155, 45.4409...","[-0.1456738126116126, -1.016331540756653, -0.4...","[-0.6682125091552749, 3.469067764282215, 1.313..."
1940,2018-12-27,"[35.824984658184114, 44.23231104423264, 42.397...","[-0.05306747483048202, -0.7654374616540056, -0...","[-0.9301653480529808, 3.3199210739135623, 1.39..."
1941,2018-12-28,"[35.924017412743005, 43.069018160977755, 48.95...","[0.029548701342652972, -0.6036722574538114, -0...","[-1.216376075744634, 3.093478469848634, 1.4717..."


,date,rsi_scores,macd_scores,sma_scores
2195,2020-01-02,"[84.6076346758042, 76.61380958575184, 75.72662...","[0.3308543935720738, 0.1434325348974408, 0.164...","[11.4599663734436, 13.111914253234858, 5.66457..."
2196,2020-01-03,"[77.7334577569535, 65.79076117784025, 78.62741...","[0.3036523508205895, 0.04914672551328625, 0.17...","[11.606843757629392, 13.343126640319838, 5.858..."
2197,2020-01-06,"[79.20988811855419, 66.82544106443787, 80.3217...","[0.29029513626207226, -0.010577155341748856, 0...","[11.755081481933594, 13.576404953002907, 6.061..."
2198,2020-01-07,"[75.98186835273052, 59.92290108750835, 83.2575...","[0.2273286146275595, -0.1585850140663405, 0.26...","[11.891444110870367, 13.722736778259275, 6.218..."
2199,2020-01-08,"[79.103844369757, 66.39392635470048, 86.182158...","[0.23187254781755096, -0.11301285815636186, 0....","[12.027988548278799, 13.892814407348624, 6.360..."
...,...,...,...,...
2443,2020-12-24,"[67.92093201746792, 61.4140453317908, 62.87192...","[0.6367954215367151, 0.8181499102977461, -1.10...","[20.250402450561523, 16.23293983459473, 64.366..."
2444,2020-12-28,"[73.64739813429696, 64.36470963724315, 63.1416...","[0.8762624898211562, 0.8841757490203559, -1.16...","[20.236367759704592, 16.021717147827133, 64.87..."
2445,2020-12-29,"[68.56496298642745, 62.47884098443792, 63.4838...","[0.8412481092093325, 0.8145809517425717, -1.22...","[20.185283451080323, 15.686962280273434, 65.42..."
2446,2020-12-30,"[65.48940943030762, 56.99493374293939, 67.5459...","[0.6758172725325955, 0.5597641207067352, -0.71...","[20.189485168457026, 15.47428779602052, 66.166..."


In [6]:
# Initialize weights with small random values or equal weights
signal_weights = np.array([1/3, 1/3, 1/3])  # [w_rsi, w_macd, w_sma]


avg_annualized_return, total_return_sum = portfolio_solver. calculate_eval_returns(
    tickers, data, df_eval, signal_weights
)


     pcost       dcost       gap    pres   dres
 0: -5.7474e-02 -1.0915e+00  1e+00  0e+00  5e+00
 1: -5.8148e-02 -1.0148e-01  4e-02  5e-17  2e-01
 2: -8.5348e-02 -9.8979e-02  1e-02  1e-16  5e-17
 3: -9.0743e-02 -9.1519e-02  8e-04  2e-16  1e-16
 4: -9.1436e-02 -9.1444e-02  8e-06  3e-18  6e-17
 5: -9.1443e-02 -9.1443e-02  8e-08  1e-16  9e-17
Optimal solution found.


AttributeError: 'Portfolio_Solver' object has no attribute 'CalculatePortfolioReturns'

In [ ]:
print(avg_annualized_return, total_return_sum)